Code for scraping from indeed.co.uk

In [1]:
#import packages
import requests, bs4, time
import pandas as pd
import os
from itertools import cycle
import traceback
from datetime import date
import pickle
import numpy as np
import datetime
import cleaning_funcs_indeed_uk as cf


In [2]:
# readin all raw pickle files

ads= pd.DataFrame(columns = ['company', 'job_title', 'salary', 'location', 'description', 'date',
       'full_description', 'other_deets', 'jobtype', 'salary_low',
       'salary_high','extraction_date'])

import os

for i in os.listdir('C:/Users/lundr/DataScienceJobs/data/indeed_raw_pickles/'):
    
    read_in = pd.read_pickle("C:/Users/lundr/DataScienceJobs/data/indeed_raw_pickles/"+i)
    ads = pd.concat([ads,read_in],ignore_index = True, axis = 0, sort=True)


In [3]:
#drop rows with identical descriptions
ads = ads.drop_duplicates(['description'])

#reset index after dropping duplicated
ads.reset_index(inplace=True)

In [4]:
ads.shape
ads[ads['salary']!='Nothing_found']

,index,company,date,description,extraction_date,full_description,job_title,jobtype,location,other_deets,salary,salary_high,salary_low
0,0,\nSecret Intelligence Service,Nothing_found,\n As an intelligence officer your focus will ...,2019-11-13,Department: Intelligence\nLocation(s): London\...,Intelligence Officer,NaN,Nothing_found,"London_£33,800 - £42,700 a year","\n£33,800 - £42,700 a year",NaN,NaN
1,1,\nNorfolk County Council,Just posted,\n This post will also involve a certain amoun...,2019-11-13,37 hours per week(30 hours per week and 22 hou...,Business and Engagement Officers,NaN,Nothing_found,"Norwich_Full-time, Part-time, Permanent_£29,63...","\n£29,636 - £31,371 a year",NaN,NaN
3,3,\n\n4Com,Nothing_found,\n Applying analytical rigour when developing ...,2019-11-13,Help us shape the future of telecoms as a Busi...,Business Intelligence Analyst,NaN,Nothing_found,"Bournemouth_£35,000 - £50,000 a year","\n£35,000 - £50,000 a year",NaN,NaN
4,5,\nHoneycomb Group,19 days ago,\n You’ll be working with the Business Intelli...,2019-11-13,Honeycomb Group is a team of social-minded bra...,Business Intelligence Assistant,NaN,Stoke-on-Trent ST4,"Stoke-on-Trent ST4_£26,250 a year","\n£26,250 a year",NaN,NaN
6,7,\n\nUniversity of London,13 days ago,\n The candidate will work for the Business In...,2019-11-13,University of London Worldwide\nUniversity of ...,Business Intelligence Analyst (3 positions),NaN,London WC1B,"London WC1B_£36,452 - £43,956 a year","\n£36,452 - £43,956 a year",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7505,18315,\nProvention,26 days ago,\n\*This project is based in Munich for 3 - 9 ...,11-11-2019,*This project is based in Munich for 3 - 9 mon...,Application Architect,NaN,Milton Keynes,Milton Keynes_Contract_£550 - £600 a day,\n£550 - £600 a day,NaN,NaN
7506,18316,\n\nUniversity of Liverpool,26 days ago,"\nMachine learning, formal verification, softw...",11-11-2019,The University of Liverpool are seeking to app...,Research Associate - Grade 7,NaN,Liverpool,Liverpool,"\n£34,805 - £40,322 a year",NaN,NaN
7517,18350,\nOpenRent,8 days ago,\nBuild your own machine (if desired!). From u...,11-11-2019,About You\nWe are looking for a software devel...,Software Engineer,NaN,London,"London_£45,000 - £65,000 a year","\n£45,000 - £65,000 a year",NaN,NaN
7518,18351,\nLaser Crystal Ltd,30+ days ago,\nOperating our 3d laser engraving machines. H...,11-11-2019,None,Apprentice / Junior 3D CAD Modeller,NaN,Poole,,"\n£12,000 - £17,000 a year",NaN,NaN


In [5]:
# clean data and create additional variables

ads['location'],ads['salary'],ads['jobtype'] = cf.clean_other_deets(ads['other_deets'])
ads['location'] = ads['location'].apply(lambda x: x.lstrip().rstrip())
ads['full_description'] = cf.clean_column(ads['full_description'])
ads['company'] = cf.clean_column(ads['company'])
ads['salary_low'], ads['salary_high'],ads['salary_type'] = cf.create_split_salary_range(ads['salary'])
ads['salary_low']=cf.clean_salary(ads['salary_low'],"£")
ads['salary_high']=cf.clean_salary(ads['salary_high'],"£")
ads['days_from_post'] = cf.create_days_from_post(ads['date'])
ads['posted_date'] = cf.create_posted_date(ads['days_from_post'],ads['extraction_date'])

ads['salary_type'] = ads['salary_type'].str.replace('n ',"")
ads['salary_type'] = ads['salary_type'].str.replace(r"[\"\',]", '')
ads['salary_type'] = ads['salary_type'].str.replace(r" ", '')

C:\Users\lundr\DataScienceJobs\Cleaning\cleaning_funcs_indeed_uk.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_df_col[i] = str(pandas_df_col[i]).strip('[]')
C:\Users\lundr\DataScienceJobs\Cleaning\cleaning_funcs_indeed_uk.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if type(a[i])!="str":a[i]=str(a[i])
C:\Users\lundr\DataScienceJobs\Cleaning\cleaning_funcs_indeed_uk.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
ads['location'].lstrip()

In [149]:
ads['salary_type'] = ads['salary_type'].str.replace('n ',"")
ads['salary_type'] = ads['salary_type'].str.replace(r"[\"\',]", '')
ads['salary_type'] = ads['salary_type'].str.replace(r" ", '')

In [6]:
# check for salary anomalies and manually correct if needed
idx = ads.loc[ads['salary_high'] != 'None']
idx[idx['salary_high']/idx['salary_low']>3]
#

,index,company,date,description,extraction_date,full_description,job_title,jobtype,location,other_deets,salary,salary_high,salary_low,salary_type,days_from_post,posted_date
2948,7972,Wokingham Borough Council,2 days ago,\n Preparation of reports and statistics (clas...,2019-11-13,Administrative Assistant Receptionist (AA )...,Administrative Assistant/Receptionist,Permanent,Wokingham,"Wokingham_Permanent_£6,000 - £19,171 a year","£6,000 - £19,171 a year",19171,6000,year,2,2019-11-11 00:00:00
3076,8264,Academic Minds,7 days ago,"\n Where applicable, the ability to use statis...",2019-11-13,Job SummaryWe are looking to recruit talented ...,Management Freelance Academic Writer,"Full-time, Part-time, Temporary, Permanent",King's Cross,"King's Cross_Full-time, Part-time, Temporary, ...",£50 - £250 a day,250,50,day,7,2019-11-06 00:00:00
4187,10683,Capita IT Resourcing,18 days ago,\nData Analysts - 3 month initial contract. Ne...,6-11-2019,Data Analysts - month initial contract Based...,Data Analyst,Contract,Sheffield,"Sheffield_Contract_£100 - £1,000 a day","£100 - £1,000 a day",1000,100,day,18,2019-05-24 00:00:00
4208,10725,Capita IT Resourcing,18 days ago,\n Data Analysts - 3 month initial contract.Ne...,6-11-2019,Data Analysts - month initial contract Based...,Data Analyst,Contract,Sheffield,"Sheffield_Contract_£100 - £1,000 a day","£100 - £1,000 a day",1000,100,day,18,2019-05-24 00:00:00
5799,14643,Cyber Tech Company,Nothing_found,\n A new business sales/business development p...,11-11-2019,Sales Executive (Cyber Tech Company)London W ...,Sales Executive,"Full-time, Permanent",London,"London_Full-time, Permanent_£30,000 - £100,000...","£30,000 - £100,000 a year",100000,30000,year,None,None
6075,15204,Cyber Tech Company,5 days ago,\nA new business sales/business development pr...,11-11-2019,Sales Executive (Cyber Tech Company)London W ...,Sales Executive,"Full-time, Permanent",London,"London_Full-time, Permanent_£30,000 - £100,000...","£30,000 - £100,000 a year",100000,30000,year,5,2019-11-06 00:00:00


In [7]:
#generate salary average after correction
ads['salary_average'] = cf.salary_average(ads['salary_low'],ads['salary_high'])



In [108]:
# force salary data to numeric

ads['salary_average'] = pd.to_numeric(ads['salary_average'],errors='coerce')
ads['salary_low'] = pd.to_numeric(ads['salary_low'],errors='coerce')
ads['salary_high'] = pd.to_numeric(ads['salary_high'],errors='coerce')


In [8]:
# generate missing columns
# drop unneeded columns
ads = ads.drop(['description', 'other_deets'], axis=1)
# job refcode
ads['ref_code'] = np.nan


In [233]:
# match locations to regions
ads['location']= ads['location'].apply(lambda x: str(x))
ads['Region'] = ads['location'].apply(lambda x: check_locations(x))

C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys


In [260]:
ads['Region'] = ads['Region'].apply(lambda x: x.split(",")[0])

In [261]:
ads[ads['Region'==''].unique()

array(['London', 'East of England', 'South West', 'West Midlands',
       'East Midlands', 'South East', 'Scotland',
       'Yorkshire and The Humber', 'North East', 'Remote', '',
       'North West', 'UK Unknown', 'Offshore', 'Wales',
       'Northern Ireland', 'East midlands'], dtype=object)

In [288]:
# some places have multiple matches, create dictionary to correct

corrections = {"brentwood" :"East",
"kingston upon thames": "London",
"richmond upon thames" : "London",
"kentish town" : "London",
"hartlepool" : "North East",
"hemel hempstead" : "East",
"chesterfield" : "East Midlands",
"rotherham" : "Yorkshire and the Humber",
"london bridge station" : "London",
"sutton-in-ashfield" : "East Midlands",
"newcastle-under-lyme" : "West Midlands",
"great chesterford" : "East",
 "bridgend cf31" : "Wales"}

for i in range(len(ads['location'])):
     if ads['location'][i].lower() in corrections.keys():
        ads.loc[ads['location']==ads['location'][i],'Region'] = corrections[ads['location'][i].lower()]

In [298]:
# i = 7234
# if ads['location'][i].lower() in corrections.keys():
#     print(corrections[ads['location'][i].lower()])
#ads[ads['location']== 'Hemel Hempstead']

Wales


In [132]:
# standardize formats for regions 
ads['Region'] = [str(x) for x in ads['Region']]
ads['Region'] = [x.lower() for x in ads['Region']]

In [178]:
# check for any Region which did not get allocated a region
ads.loc[ads['Region']=='0.0','Region'] = np.nan


In [346]:
#find rows with no salaries


In [56]:
#ads.to_pickle("C:/Users/lundr/DataScienceJobs/data/indeeduk__cleaned_13_11_19")

In [329]:
ads.shape[0]

7524

In [331]:
for x in range(ads.shape[0]):
    for y in range(ads.shape[1]):
        if ads.iloc[x,y] =='None' or ads.iloc[x,y] =='Nothing_found':
            ads.iloc[x,y] = np.nan

In [330]:
print(x)
print(y)

20
20


In [333]:

table_data = ads[ads['salary_type']=='year']
#table_data.reset_index(inplace=True)
table_data.groupby(by='Region').agg({'salary_average' : ['count','median','mean','max','min']}).reset_index().sort_values(by =('salary_average', 'mean'))

Region salary_average                                   \
                                      count   median          mean       max   
3              East midlands              1  19806.0  19806.000000   19806.0   
8           Northern Ireland              3  18000.0  21166.666667   27500.0   
2              East Midlands             96  33724.5  33427.906250   70000.0   
14                     Wales             47  31000.0  33537.893617   55000.0   
6                 North East             34  30500.0  33928.117647  100000.0   
16  Yorkshire and The Humber            113  32831.0  35013.823009  100000.0   
7                 North West            142  33834.0  35220.007042   90000.0   
12                South West            151  32012.0  35479.496689   80000.0   
10                  Scotland             51  31250.0  36085.588235   80000.0   
4            East of England            154  35000.0  36708.668831   87500.0   
1                       East              7  32500.0  37042.857143   60000.0   
15             West Midlands            177  35632.0  37571.966102   85000.0   
11                South East            272  35000.0  38849.117647  110000.0   
9                     Remote              9  37500.0  39222.222222   65000.0   
13                UK Unknown             71  36569.0  43104.014085  125000.0   
0                                       196  45000.0  48733.173469  125000.0   
5                     London            943  47500.0  53071.317073  152500.0   

             
        min  
3   19806.0  
8   18000.0  
2    9125.0  
14  19000.0  
6   17007.0  
16  10791.0  
7   12191.0  
12  15487.0  
10  18647.0  
4   16436.0  
1   23000.0  
15  18157.0  
11  12585.0  
9   23000.0  
13  11402.0  
0   18172.0  
5   14442.0

In [150]:
ads['salary_type'].unique

array(['year', 'None', 'day', 'hour', 'week', 'month'], dtype=object)

In [168]:
table_data['salary_average']=pd.to_numeric(table_data['salary_average'])
table_data.dtypes

C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


index                 int64
company              object
date                 object
extraction_date      object
full_description     object
job_title            object
jobtype              object
location             object
salary               object
salary_high         float64
salary_low          float64
salary_type          object
days_from_post       object
posted_date          object
salary_average      float64
industry            float64
education           float64
career              float64
country              object
ref_code            float64
Region               object
dtype: object

In [61]:
np.mean(table_data['salary_average'])

39204.90817064353

In [169]:
table_data[table_data['salary_low']>2000].groupby(['Region']).agg({'salary_average' : ['sum']}).reset_index()

,Region,salary_average
,,sum
0,0.0,7224610.0
1,east midlands,2867602.0
2,east of england,5807767.0
3,isle of man,75000.0
4,london,49782311.0
5,north east,1173608.0
6,north west,5528399.0
7,northern ireland,63500.0
8,remote,353000.0


In [76]:
table_data[table_data['salary_low']>2000].groupby(['Region']).mean().reset_index()

,Region,index,industry,education,career,ref_code
0,0,11557.141844,NaN,NaN,NaN,NaN
1,Belfast,10341.000000,NaN,NaN,NaN,NaN
2,East,8414.854167,NaN,NaN,NaN,NaN
3,East Midlands,8727.572917,NaN,NaN,NaN,NaN
4,East midlands,9396.000000,NaN,NaN,NaN,NaN
5,East of England,9411.188525,NaN,NaN,NaN,NaN
6,Isle of Man,9155.000000,NaN,NaN,NaN,NaN
7,Kent,6292.666667,NaN,NaN,NaN,NaN
8,London,10014.237234,NaN,NaN,NaN,NaN
9,London,18272.000000,NaN,NaN,NaN,NaN
